In [1]:
import pandas as pd
url= '/Users/amirrezakamkar/Desktop/Works_in_progress/Ironhack/3_3rd_week/Project/first_project/data/clean/merged_clean_owen2.csv'
df= pd.read_csv(url)

In [2]:
df.columns

Index(['steamId', 'name', 'release_date', 'copies_sold', 'price_USD',
       'price_local', 'revenue', 'avgPlaytime', 'publisherClass', 'publishers',
       'developers', 'required_age', 'is_free', 'description',
       'about the game', 'languages', 'language_count', 'genres', 'categories',
       'estimated owners', 'peak ccu', 'metacritic score', 'review_score',
       'positive', 'negative', 'recommendations', 'average playtime forever',
       'average playtime two weeks', 'median playtime forever',
       'median playtime two weeks', 'Early Access', 'Strategy', 'Casual',
       'Simulation', 'RPG', 'Racing', 'Massively Multiplayer', 'Indie',
       'Action', 'Free To Play', 'Adventure', 'Sports', 'Single-player',
       'Multi-player', 'Steam Achievements', 'In-App Purchases',
       'Partial Controller Support', 'Steam Cloud', 'Family Sharing',
       'Full controller support', 'Steam Trading Cards'],
      dtype='object')

In [3]:
df.head()

,steamId,name,release_date,copies_sold,price_USD,price_local,revenue,avgPlaytime,publisherClass,publishers,...,Sports,Single-player,Multi-player,Steam Achievements,In-App Purchases,Partial Controller Support,Steam Cloud,Family Sharing,Full controller support,Steam Trading Cards
0,2315690,WWE 2K24,2024-03-07,165301,99.99,£24.99,8055097.0,42.365140,AAA,2K,...,True,True,True,True,True,True,True,True,False,False
1,2291060,EARTH DEFENSE FORCE 6,2024-07-25,159806,59.99,£49.99,7882151.0,29.651061,Indie,D3PUBLISHER,...,False,True,True,True,False,False,True,True,True,False
2,1575940,Sins of a Solar Empire II,2024-08-15,214192,49.99,£41.99,7815247.0,12.452593,Indie,Stardock Entertainment,...,False,True,True,False,False,False,True,True,False,True
3,1859910,Legend of Mortal,2024-06-14,440998,19.99,£10.00,7756399.0,24.797817,Indie,"Paras Games, Obb Studio Inc.",...,False,True,False,True,False,False,True,True,True,False
4,1875830,Shin Megami Tensei V: Vengeance,2024-06-13,141306,59.99,CDN$ 55.99,7629252.0,34.258496,AA,SEGA,...,False,True,False,True,False,False,True,True,True,False


In [7]:
df.isnull().sum()

steamId                         0
name                            0
release_date                    0
copies_sold                     0
price_USD                       0
price_local                    90
revenue                         0
avgPlaytime                     0
publisherClass                  0
publishers                      0
developers                      0
required_age                    0
is_free                         2
description                     2
about the game                229
languages                       2
language_count                  2
genres                          2
categories                      2
estimated owners              222
peak ccu                      222
metacritic score              222
review_score                    0
positive                      222
negative                      222
recommendations               222
average playtime forever      222
average playtime two weeks    222
median playtime forever       222
median playtim

In [ ]:
df.price_USD.mean()

In [ ]:
#finding the average price to divide the dataset into two groups, higher than average and lower than average.
print(df.price_USD.mean())

# taking the average as the threshold and comparing the review_score average of the two groups.
threshold = 17.5
df_expensive = df[df['price_USD'] > threshold]
print("the average review score of more expensive games is ", df_expensive.review_score.mean())

threshold = 17
df_expensive = df[df['price_USD'] < threshold]
print("the average review score of cheaper games is ", df_expensive.review_score.mean())

# using median to divide the dataset into two groups. in this case median is a better choice than mean as the very high and very low amounts
#doesn't change it considerably.
print(df.review_score.median())

# taking the median as the threshold and comparing the revenue average for the two groups.
review_threshold = 83
df_high_review = df[df['review_score'] > review_threshold]
print('the median revenue of games with higher than median review score is ', df_high_review.revenue.mean())

review_threshold = 83
df_high_review = df[df['review_score'] < review_threshold]
print('the average revenue of games with lower than median review score is ', df_high_review.revenue.mean())

In [ ]:
df.price_USD.mean()

In [ ]:
df['Sports_revenue'] = df.apply(lambda x: x['revenue'] if x['Sports'] == True else None, axis=1)
df.Sports_revenue.mean()

In [ ]:
import numpy as np

In [ ]:
game_types = ['Strategy', 'Casual', 'Simulation', 'RPG', 'Racing', 'Massively Multiplayer', 'Action', 'Free To Play', 'Adventure', 'Sports']

# Create new revenue columns for each game type
for game_type in game_types:
    df[f'{game_type}_revenue'] = df.apply(lambda x: x['revenue'] if x[game_type] else np.nan, axis=1)

# Calculate means for the new revenue columns
mean_revenues = {f'{game_type}_revenue': df[f'{game_type}_revenue'].mean() for game_type in game_types}

print(mean_revenues)

In [ ]:
N = 5  # Number of top scores to retrieve
top_n = sorted(mean_revenues.items(), key=lambda item: item[1], reverse=True)[:N]
print("Top N Scores:", top_n)

In [ ]:
indie_df = df[(df['publisherClass'] == 'Indie')]

melt_by_genre = pd.melt(indie_df, id_vars=['name', 'revenue', 'copies_sold'],
                        value_vars=['Simulation', 'RPG', 'Racing', 'Massively Multiplayer', 'Action',
                                     'Free To Play', 'Adventure', 'Sports'],
                        var_name='genre', value_name='is_in_genre')

melt_by_genre = melt_by_genre[melt_by_genre['is_in_genre'] == True]

genre_summary = melt_by_genre.groupby('genre').agg(
    count=('name', 'count'),
    total_revenue=('revenue', 'sum'),
    avg_revenue_per_game= ('revenue', 'mean'),
    total_copies_sold = ('copies_sold', 'sum'),
    min_revenue=('revenue', 'min'),                  
    Q1_revenue=('revenue', lambda x: x.quantile(0.25)),  
    median_revenue=('revenue', 'median'),            
    Q3_revenue=('revenue', lambda x: x.quantile(0.75)),
    max_revenue=('revenue', 'max')     
).reset_index()

genre_summary['avg_rev_per_copy_sold'] = genre_summary['total_revenue'] / genre_summary['total_copies_sold']

genre_summary = genre_summary.sort_values(by=['total_revenue'], ascending=[False])

genre_summary

In [ ]:
indie_by_genre = pd.melt(indie_df, id_vars=['name', 'revenue', 'copies_sold'],
                        value_vars=['Simulation', 'RPG', 'Racing', 'Action',
                                     'Adventure', 'Sports'],
                        var_name='genre', value_name='is_in_genre')

indie_by_genre = indie_by_genre[indie_by_genre['is_in_genre'] == True]

indie_by_genre = indie_by_genre[['name', 'revenue', 'copies_sold', 'genre']]

indie_by_genre

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df = indie_by_genre[['name', 'revenue', 'copies_sold']].copy()
df = df.drop_duplicates(subset='name')

# Histogram for Revenue and Copies Sold (log scale for better visualization)
sns.histplot(df['revenue'], kde=True, bins=30, log_scale=True)
plt.title('Distribution of Game Revenue')
plt.savefig(r'..\figures\game_revenue_distribution.png', format="png", dpi=300, bbox_inches='tight', transparent=True)
plt.show()

sns.histplot(df['copies_sold'], kde=True, bins=30, log_scale=True)
plt.title('Distribution of Copies Sold')
plt.savefig(r'..\figures\game_copies_sold_distribution.png', format="png", dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
df = indie_by_genre[['name', 'genre', 'revenue', 'copies_sold']].copy()

genre_performance = df.groupby('genre').agg(
    total_revenue=('revenue', 'sum'),
    total_copies_sold=('copies_sold', 'sum'),
    count=('name', 'nunique')  # count of unique games per genre
).sort_values(by='total_revenue', ascending=False)

# Bar plots for total revenue and copies sold by genre
plt.figure(figsize=(12, 6))
sns.barplot(x=genre_performance.index, y=genre_performance['total_revenue'], palette='Set2')
plt.title('Total Revenue by Genre')
plt.xlabel('Genre')
plt.ylabel('Total Revenue')
plt.xticks(rotation=45)
plt.savefig(r'..\figures\indie_total_rev_by_genre.png', format="png", dpi=300, bbox_inches='tight', transparent=True)
plt.show()